# LightGBM Baseline Notebook
reference: https://www.kaggle.com/code/greysky/lightgbm-starter-with-us-map-lb-1-0871

In [1]:
import os

import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna

from utils import lagging_on, transpose_columns

pd.set_option('display.max_columns', 50)
# folder path
HOST_PROVIDED = 'data/host_provided'
EXTERNAL = 'data/external'
SAVE_PATH = 'data/ready'
# train data metadata
date_col = 'first_day_of_month'
cat_cols = ['county', 'state']
target = 'microbusiness_density'
idx = 'row_id'
# feature engineering parameter
LAGS = 12

c:\Users\LM195WL\OneDrive - EY\Desktop\Python Scripts\kaggle_godaddy_forecasting\kaggle_venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Feature Engineering

### Load host-provided datasets
- train.csv
- test.csv
- sample_submission.csv
- census_starter.csv

In [2]:
df_train = pd.read_csv(os.path.join(HOST_PROVIDED, 'train.csv'), index_col=idx)
df_test = pd.read_csv(os.path.join(HOST_PROVIDED, 'test.csv'), index_col=idx)
df_subm = pd.read_csv(os.path.join(HOST_PROVIDED, 'sample_submission.csv'), index_col=idx)
df_census = pd.read_csv(os.path.join(HOST_PROVIDED, 'census_starter.csv'), index_col="cfips")

### Preprocessing

In [3]:
# get cfips - (state - county) mapping
state_dict = df_train[['cfips', 'state', 'county']]
state_dict = state_dict.set_index('cfips')
state_dict = state_dict.drop_duplicates()
state_dict = state_dict.to_dict()
# map test df's cfips to state and county, then concatenate train / test set for feature engineering
df_test['state'] = df_test['cfips'].map(state_dict['state'])
df_test['county'] = df_test['cfips'].map(state_dict['county'])
df_all = pd.concat([df_train, df_test], axis=0)
# convert date column to datetime type
df_all[date_col] = pd.to_datetime(df_all[date_col])
# separate date time to year and month, add relative month to the first month as 'scale' column
df_all['year'] = df_all[date_col].dt.year
df_all['month'] = df_all[date_col].dt.month
df_all['scale'] = (df_all[date_col] - df_all[date_col].min()).dt.days
df_all['scale'] = df_all['scale'].factorize()[0]
# drop and convert
df_all = df_all.drop(columns=[date_col])
df_all[cat_cols] = df_all[cat_cols].astype('category')

### Generate lag data
generate two time series on the following columns, the time span is 12 (LAGS) months
- microbusiness_density
- active

In [4]:
# add lag data on "microbusiness_density" and "active" columns
df_all = lagging_on(df_all, groupby="cfips", columns=["microbusiness_density", "active"], n_lag=LAGS)
# drop the first month (lagging data columns are empty)
df_all = df_all[df_all['scale'] != 0]


### Join the dataset with population_data.csv

In [5]:
df_population = pd.read_csv(os.path.join(EXTERNAL, 'population_data.csv'))
# since the host use 2-year-lagged data
df_population.year = df_population.year + 2000 + 2
df_all = pd.merge(df_all.reset_index(), df_population, how="left", left_on=['cfips', 'year'], right_on=['cfips', 'year']).drop(columns=["Geography", "Geographic Area Name"])

### Aggregate neighbour county's population
sum aggregation for:
- Total population
- Under 18 years

mean aggregation for:
- median_hh_inc

In [6]:

# load data
df_neighbours = pd.read_csv(os.path.join(EXTERNAL, 'county-neighbours.csv'))
df_population = pd.read_csv(os.path.join(EXTERNAL, 'population_data.csv'))
df_neighbours = df_neighbours.rename(columns={"Neighbour county code": "cfips"})
# join two df
df_neighbours_merge = pd.merge(df_neighbours, df_population, how="left", left_on=['cfips'], right_on=['cfips'])
income_columns = ["median_hh_inc_2017", "median_hh_inc_2018", "median_hh_inc_2019", "median_hh_inc_2020", "median_hh_inc_2021"]
# join with host-provided census data's income related columns
df_neighbours_merge = pd.merge(df_neighbours_merge, df_census[income_columns].reset_index(), how="left", left_on=['cfips'], right_on=['cfips']).dropna()
# sum aggregate
agg_neighbour = df_neighbours_merge.groupby(["Countycode", "year"])[["Total population", "Under 18 years"]].sum()
agg_neighbour =agg_neighbour.rename(columns={"Total population": "sum_neighbour_p", "Under 18 years": "sum_neighbour_u18"})
# mean aggregate
mean_neighbour_inc = df_neighbours_merge.groupby(["Countycode"])[income_columns].mean()
# transpose table
hh_inc_f = lambda x:float(x.replace("median_hh_inc_20", ""))
agg_neighbour["mean_neighbour_median_hh_inc"] = transpose_columns(mean_neighbour_inc, col_f=hh_inc_f)
agg_neighbour = agg_neighbour.reset_index().rename(columns={"Countycode": "cfips"})
agg_neighbour.year = agg_neighbour.year + 2000 + 2
df_all = pd.merge(df_all, agg_neighbour, how="left", left_on=["cfips", "year"], right_on=["cfips", "year"])

In [7]:
agg_neighbour

,cfips,year,sum_neighbour_p,sum_neighbour_u18,mean_neighbour_median_hh_inc
0,1001,2019.0,403031.0,95232.0,40975.400000
1,1001,2020.0,402348.0,94665.0,42990.800000
2,1001,2021.0,401368.0,94004.0,44472.800000
3,1001,2022.0,400244.0,93386.0,46675.800000
4,1001,2023.0,410571.0,96299.0,48653.800000
...,...,...,...,...,...
15695,56045,2019.0,213771.0,50830.0,57808.571429
15696,56045,2020.0,214664.0,50380.0,58570.571429
15697,56045,2021.0,216106.0,50564.0,60684.285714
15698,56045,2022.0,217815.0,50411.0,63240.285714


### Transpose census_starter.csv and index by their 'year'
transpose the following columns:
- pct_bb
- pct_college
- pct_foreign_born
- pct_it_workers
- median_hh_inc

In [8]:
# transpose and create a new dataframe
pct_bb_cols = ["pct_bb_2017", "pct_bb_2018", "pct_bb_2019", "pct_bb_2020", "pct_bb_2021"]
pct_bb_f = lambda x:float(x.replace("pct_bb_20", ""))
df_census_transposed = transpose_columns(df_census, columns=pct_bb_cols, col_name="year", col_f=pct_bb_f)
df_census_transposed = pd.DataFrame(df_census_transposed, columns=["pct_bb"])
pct_college_cols = ["pct_college_2017", "pct_college_2018", "pct_college_2019", "pct_college_2020", "pct_college_2021"]
pct_college_f = lambda x:float(x.replace("pct_college_20", ""))
df_census_transposed["pct_college"] = transpose_columns(df_census, columns=pct_college_cols, col_name="year", col_f=pct_college_f)
pct_foreign_born_cols = ["pct_foreign_born_2017", "pct_foreign_born_2018", "pct_foreign_born_2019", "pct_foreign_born_2020", "pct_foreign_born_2021"]
pct_foreign_born_f = lambda x:float(x.replace("pct_foreign_born_20", ""))
df_census_transposed["pct_foreign_born"] = transpose_columns(df_census, columns=pct_foreign_born_cols, col_name="year", col_f=pct_foreign_born_f)
pct_it_workers_cols = ["pct_it_workers_2017", "pct_it_workers_2018", "pct_it_workers_2019", "pct_it_workers_2020", "pct_it_workers_2021"]
pct_it_workers_f = lambda x:float(x.replace("pct_it_workers_20", ""))
df_census_transposed["pct_it_workers"] = transpose_columns(df_census, columns=pct_it_workers_cols, col_name="year", col_f=pct_it_workers_f)
median_hh_inc_cols = ["median_hh_inc_2017", "median_hh_inc_2018", "median_hh_inc_2019", "median_hh_inc_2020", "median_hh_inc_2021"]
median_hh_inc_f = lambda x:float(x.replace("median_hh_inc_20", ""))
df_census_transposed["median_hh_inc"] = transpose_columns(df_census, columns=median_hh_inc_cols, col_name="year", col_f=median_hh_inc_f)
df_census_transposed.reset_index().to_csv(os.path.join(EXTERNAL, "census_transposed.csv"), index=False)
# join the dataset with the transposed census dataset
df_census_transposed = df_census_transposed.reset_index()
df_census_transposed.year = df_census_transposed.year + 2000 + 2
df_all = pd.merge(df_all, df_census_transposed, how="left", left_on=["cfips", "year"], right_on=["cfips", "year"])

In [9]:
df_census_transposed

,cfips,year,pct_bb,pct_college,pct_foreign_born,pct_it_workers,median_hh_inc
0,1001,2019.0,76.6,14.5,2.1,1.3,55317.0
1,1001,2020.0,78.9,15.9,2.0,1.1,58786.0
2,1001,2021.0,80.6,16.1,2.3,0.7,58731.0
3,1001,2022.0,82.7,16.7,2.3,0.6,57982.0
4,1001,2023.0,85.5,16.4,2.1,1.1,62660.0
...,...,...,...,...,...,...,...
15703,56045,2019.0,71.1,14.1,3.8,0.6,59605.0
15704,56045,2020.0,73.3,13.5,4.1,0.6,52867.0
15705,56045,2021.0,76.8,13.4,1.7,0.0,57031.0
15706,56045,2022.0,79.7,12.7,2.3,0.0,53333.0


### (Optional) Add masked dataset replica to train a more robust model

In [ ]:
# (Optional) add masked replica
from utils import random_masking
columns = [f"lag_{i+1}_microbusiness_density" for i in np.arange(LAGS)] + [f"lag_{i+1}_active" for i in np.arange(LAGS)]
df_all_replica = random_masking(df_all, columns=columns, n_rep=10, threshold=0.8)

### Feature set ready

In [10]:
df_all = df_all.set_index("row_id")

## Explore LightGBM hyperparmeters with Optuna

### Define training trick and loss function

In [11]:
# training trick
def to_percent(X, y):
    yhat = y / X['lag_1_microbusiness_density']
    yhat[X['lag_1_microbusiness_density'] == 0] = 0 # denominator cannot be 0
    return yhat

def from_percent(X, yhat):
    y = yhat * X[f'lag_1_microbusiness_density']
    return y
# loss function
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    pos_ind = (y_true != 0) | (y_pred != 0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    return 100 * np.mean(smap)

In [12]:
SMAPE_ENABLED = True
def lgb_objective(trial):
    # define hyperparameters and searching range
    params = {
        'n_iter'           : 200,
        'verbosity'        : -1,
        'objective'        : 'l1',
        'random_state'     : 42,
        'extra_trees'      : True,
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode' : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'max_depth'        : trial.suggest_int('max_depth', 3, 10),
        'learning_rate'    : trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'lambda_l1'        : trial.suggest_float('lambda_l1', 1e-2, 10.0),
        'lambda_l2'        : trial.suggest_float('lambda_l2', 1e-2, 10.0),
        'num_leaves'       : trial.suggest_int('num_leaves', 8, 1024),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 5, 250),}
    # LightGBM regression model
    model  = lgb.LGBMRegressor(**params)
    # training data pipeline
    X, y   = df_all.drop(columns=[target, "active"]), df_all[target]
    # split train / validation set
    train_times = list(range(38))
    X_train = X[X['scale'].isin(train_times)]
    y_train = y[X['scale'].isin(train_times)]
    valid_times = [38]
    X_valid = X[X['scale'].isin(valid_times)]
    y_valid = y[X['scale'].isin(valid_times)]
    # training
    if SMAPE_ENABLED:
        y_train = to_percent(X=X_train, y=y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    if SMAPE_ENABLED:
        y_pred = from_percent(X=X_valid, yhat=y_pred)
    return smape(y_valid, y_pred)

In [13]:
study = optuna.create_study(direction='minimize', study_name='Regressor')
study.optimize(lgb_objective, n_trials=50)

[I 2023-02-20 23:35:47,105] A new study created in memory with name: Regressor
c:\Users\LM195WL\OneDrive - EY\Desktop\Python Scripts\kaggle_godaddy_forecasting\kaggle_venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=145, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=145
[LightGBM] [Warning] lambda_l1 is set=6.826332093262305, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.826332093262305
[LightGBM] [Warning] lambda_l2 is set=3.404079767774333, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.404079767774333
[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200


[I 2023-02-20 23:35:51,253] Trial 0 finished with value: 1.0682763749613446 and parameters: {'colsample_bytree': 0.7074145505094599, 'colsample_bynode': 0.3136684886107868, 'max_depth': 10, 'learning_rate': 0.0342958946166189, 'lambda_l1': 6.826332093262305, 'lambda_l2': 3.404079767774333, 'num_leaves': 556, 'min_data_in_leaf': 145}. Best is trial 0 with value: 1.0682763749613446.
c:\Users\LM195WL\OneDrive - EY\Desktop\Python Scripts\kaggle_godaddy_forecasting\kaggle_venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=102, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=102
[LightGBM] [Warning] lambda_l1 is set=8.34983438617155, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.34983438617155
[LightGBM] [Warning] lambda_l2 is set=8.97548760916854, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8.97548760916854
[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200


[I 2023-02-20 23:35:53,904] Trial 1 finished with value: 1.0753098811841098 and parameters: {'colsample_bytree': 0.4360827113312845, 'colsample_bynode': 0.5544234855373612, 'max_depth': 10, 'learning_rate': 0.013449885118313479, 'lambda_l1': 8.34983438617155, 'lambda_l2': 8.97548760916854, 'num_leaves': 32, 'min_data_in_leaf': 102}. Best is trial 0 with value: 1.0682763749613446.
c:\Users\LM195WL\OneDrive - EY\Desktop\Python Scripts\kaggle_godaddy_forecasting\kaggle_venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[I 2023-02-20 23:37:46,807] Trial 47 finished with value: 1.0699405595100902 and parameters: {'colsample_bytree': 0.15983750714062317, 'colsample_bynode': 0.6454260317790604, 'max_depth': 9, 'learning_rate': 0.09466545535410104, 'lambda_l1': 3.597164171869299, 'lambda_l2': 5.952842405725468, 'num_leaves': 872, 'min_data_in_leaf': 48}. Best is trial 44 with value: 1.0661006696128192.
c:\Users\LM195WL\OneDrive - EY\Desktop\Python Scripts\kaggle_godaddy_forecasting\kaggle_venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=108, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=108
[LightGBM] [Warning] lambda_l1 is set=4.949683443818313, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.949683443818313
[LightGBM] [Warning] lambda_l2 is set=6.474362496376458, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.474362496376458
[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200


[I 2023-02-20 23:37:48,830] Trial 48 finished with value: 1.0656925661973748 and parameters: {'colsample_bytree': 0.22202173638227363, 'colsample_bynode': 0.8252477342750326, 'max_depth': 8, 'learning_rate': 0.07857132545866048, 'lambda_l1': 4.949683443818313, 'lambda_l2': 6.474362496376458, 'num_leaves': 120, 'min_data_in_leaf': 108}. Best is trial 48 with value: 1.0656925661973748.
c:\Users\LM195WL\OneDrive - EY\Desktop\Python Scripts\kaggle_godaddy_forecasting\kaggle_venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=146, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=146
[LightGBM] [Warning] lambda_l1 is set=2.355951653012947, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.355951653012947
[LightGBM] [Warning] lambda_l2 is set=6.696223908197568, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.696223908197568
[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200


[I 2023-02-20 23:37:50,946] Trial 49 finished with value: 1.0689018382853406 and parameters: {'colsample_bytree': 0.1352648873893217, 'colsample_bynode': 0.9271095508393492, 'max_depth': 10, 'learning_rate': 0.07927062251835817, 'lambda_l1': 2.355951653012947, 'lambda_l2': 6.696223908197568, 'num_leaves': 142, 'min_data_in_leaf': 146}. Best is trial 48 with value: 1.0656925661973748.


In [14]:
study.best_value

1.0656925661973748

In [15]:
study.best_params

{'colsample_bytree': 0.22202173638227363,
 'colsample_bynode': 0.8252477342750326,
 'max_depth': 8,
 'learning_rate': 0.07857132545866048,
 'lambda_l1': 4.949683443818313,
 'lambda_l2': 6.474362496376458,
 'num_leaves': 120,
 'min_data_in_leaf': 108}

## Retrain LightGBM model with the optimal hyperparameter set

In [16]:
params = {
    'n_iter'           : 200,
    'verbosity'        : -1,
    'objective'        : 'l1',
    'random_state'     : 42,
    'extra_trees'      : True,
    'colsample_bytree': 0.22202173638227363,
    'colsample_bynode': 0.8252477342750326,
    'max_depth': 8,
    'learning_rate': 0.07857132545866048,
    'lambda_l1': 4.949683443818313,
    'lambda_l2': 6.474362496376458,
    'num_leaves': 120,
    'min_data_in_leaf': 108
    }
model = lgb.LGBMRegressor(**params)
X, y = df_all.drop(columns=[target, "active"]), df_all[target]
# include validation set in the training set
train_times = list(range(39))
X_train = X[X['scale'].isin(train_times)]
y_train = y[X['scale'].isin(train_times)]
# test set
X_test, y_test = X[y.isnull()], y[y.isnull()]
# retrain and predict
if SMAPE_ENABLED:
        y_train = to_percent(X=X_train, y=y_train)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
if SMAPE_ENABLED:
    y_pred = from_percent(X=X_test, yhat=y_pred)

c:\Users\LM195WL\OneDrive - EY\Desktop\Python Scripts\kaggle_godaddy_forecasting\kaggle_venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=108, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=108
[LightGBM] [Warning] lambda_l1 is set=4.949683443818313, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.949683443818313
[LightGBM] [Warning] lambda_l2 is set=6.474362496376458, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.474362496376458
[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200


## Create submission based on the prediction

In [17]:
df_subm.loc[X_test.index, target] = y_pred
df_subm.fillna(0).to_csv(os.path.join(SAVE_PATH, "submission_6.csv"))